In [1]:
%%bash
# download librispeech
wget -c https://us.openslr.org/resources/12/test-clean.tar.gz
tar zxf test-clean.tar.gz
# make a 30s audio
ffmpeg -f concat -safe 0 -i <(for f in $(find $LibriSpeech|grep flac$|sort); do echo "file '$PWD/$f'"; done) -t 1800 -y test_30mins.mp3
# install deps
pip install transformers

--2024-06-13 02:43:13--  https://us.openslr.org/resources/12/test-clean.tar.gz
Resolving us.openslr.org (us.openslr.org)... 46.101.158.64
Connecting to us.openslr.org (us.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

find: ‘./lost+found’: Permission denied
ffmpeg version 4.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 11.2.0 (Anaconda gcc)
  configuration: --prefix=/opt/conda/conda-bld/ffmpeg_1691625407851/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeh --cc=/opt/conda/conda-bld/ffmpeg_1691625407851/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264

In [9]:
from IPython.display import Audio
# Audio("test_30mins.mp3")

## whisper
https://huggingface.co/openai/whisper-large-v3

In [3]:
import time
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=False, use_safetensors=True,
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# Chunked Long-Form

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=25,
    stride_length_s=5,
    batch_size=48,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

# for warmup
pipe("./LibriSpeech/test-clean/61/70968/61-70968-0000.flac")

print("Starting Chunked Long-Form")
tic = time.time()
mp3 = "./test_30mins.mp3"
result = pipe(mp3)
print(result["text"][:1000])
print("using time:", time.time() - tic)

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


Starting Chunked Long-Form
 He hoped there would be stew for dinner, turnips and carrots and bruised potatoes and fat mutton pieces to be ladled out in thick, peppered, flour-fattened sauce. "'Stuff it into you,' his belly counseled him. After early nightfall the yellow lamps would light up here and there the squalid quarter of the brothels. "'Hello, Bertie. Any good in your mind? Number ten! Fresh Nelly is waiting on you. Good night, husband. The music came nearer, and he recalled the words—the words of Shelley's fragment upon the moon wandering companionless, pale for weariness. The dull light fell more faintly upon the page whereon another equation began to unfold itself slowly and to spread abroad its widening tail. A cold lucid indifference reigned in his soul. The chaos in which his ardor extinguished itself was a cold, indifferent knowledge of himself. At most, by an alms given to a beggar whose blessing he fled from, he might hope wearily to win for himself some measure of actu

In [5]:
# Sequential Long-Form

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

# for warmup
pipe("./LibriSpeech/test-clean/61/70968/61-70968-0000.flac")

print("Starting Sequential Long-Form")
tic = time.time()
mp3 = "./test_30mins.mp3"
result = pipe(mp3)
print(result["text"][:1000])
print("using time:", time.time() - tic)

Starting Sequential Long-Form
 He hoped there would be stew for dinner, turnips and carrots and bruised potatoes and fat mutton pieces to be ladled out in thick, peppered, flour-fattened sauce. Stuff it into you, his belly counseled him. After early nightfall, the yellow lamps would light up here and there, the squalid quarter of the brothels. Hello, Bertie. Any good in your mind? Number ten. Fresh Nelly is waiting on you. Good night, husband. The music came nearer, and he recalled the words, the words of Shelley's fragment upon the moon wandering companionless, pale for weariness. The dull light fell more faintly upon the page, whereon another equation began to unfold itself slowly and to spread abroad its widening tail. A cold, lucid indifference reigned in his soul. The chaos in which his ardor extinguished itself was a cold, indifferent knowledge of himself. At most, by an alms given to a beggar whose blessing he fled from, he might hope wearily to win for himself some measure of a

## distil-whisper
https://huggingface.co/distil-whisper/distil-large-v3

In [6]:
import time
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "distil-whisper/distil-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=False, use_safetensors=True,
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/4.25k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
# Chunked Long-Form

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=25,
    stride_length_s=5,
    batch_size=48,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

# for warmup
pipe("./LibriSpeech/test-clean/61/70968/61-70968-0000.flac")

print("Starting Chunked Long-Form")
tic = time.time()
mp3 = "./test_30mins.mp3"
result = pipe(mp3)
print(result["text"][:1000])
print("using time:", time.time() - tic)

Starting Chunked Long-Form
 He hoped there would be stew for dinner, turnips and carrots and bruised potatoes and fat mutton pieces to be ladled out in thick, peppered, flour-fattened sauce. Stuff it into you, his belly counselled him. After early nightfall the yellow lamps would light up here and there the squalid quarter of the brothels. Hello, Bertie! Any good in your mind? Number ten. Fresh Nelly is waiting on you. Good night, husband. The music came nearer, and he recalled the words, the words of Shelley's fragment upon the moon wandering companionless, pale for weariness. The dull light fell more faintly upon the page, whereon another equation began to unfold itself slowly and to spread abroad its widening tail. A cold, lucid indifference reigned in his soul. The chaos in which his ardor extinguished itself was a cold, indifferent knowledge of himself. At most, by an alms given to a beggar whose blessing he fled from, he might hope wearily to win for himself some measure of actua

In [8]:
# Sequential Long-Form

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

# for warmup
pipe("./LibriSpeech/test-clean/61/70968/61-70968-0000.flac")

print("Starting Sequential Long-Form")
tic = time.time()
mp3 = "./test_30mins.mp3"
result = pipe(mp3)
print(result["text"][:1000])
print("using time:", time.time() - tic)

Starting Sequential Long-Form
 He hoped there would be stew for dinner, turnips and carrots and bruised potatoes and fat mutton pieces to be ladled out in thick, peppered, flour-fattened sauce. Stuff it into you, his belly counseled him. After early nightfall, the yellow lamps would light up here and there the squalid quarter of the brothels. Hello, Bertie. Any good in your mind? Number ten. Fresh Nelly is waiting on you. Good night, husband. The music came nearer. and he recalled the words, the words of Shelley's fragment upon the moon wandering companionless, pale for weariness. The dull light fell more faintly upon the page, whereon another equation began to unfold itself slowly and to spread abroad its widening tale. A cold, lucid indifference reigned in his soul. The chaos in which his ardor extinguished itself was a cold, indifferent knowledge of himself. At a cold, indifferent knowledge of himself. At most, by an alms given to a beggar whose blessing he fled from, he might hope 